In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In [2]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base

base = reset_base()

def get_cords(frac):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(n)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

cords = get_cords(0.05)
results = []

def evaluate(method, df, t):
       
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train(cords)
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print('')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')
    
    results.append([method, r2, rmse, still_missing, t])

#getting train data by changing randomly chosen values to NaN
def reset_train(cords):
    train = base.copy()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train   

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)


In [3]:
def knn_imputer(df, n):
    
    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    knn_imp = KNNImputer(n_neighbors=n)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)

    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [4]:
ns = np.arange(2, 11)
print(ns)
for n in ns:
    n= int(n)
    base = reset_base()
    train = reset_train(cords)

    t0 = time.time()

    df = knn_imputer(train, n)

    t1 = time.time()

    t = t1-t0

    #df.to_csv('additional_data/imputed_sets/ice3.csv')
    evaluate(f'KNN with n={n}', df, t)

[ 2  3  4  5  6  7  8  9 10]
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9013044108673886, rmse: 0.12587006387706304
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9551667453634992, rmse: 0.0848347363262953
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9700605815538997, rmse: 0.06932585920533507
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9661827171754612, rmse: 0.07367886643660555
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9541147332596787, rmse: 0.08582428836677053
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9150813796939261, rmse: 0.11675488097554382
Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.8681749221856339, rmse: 0.145

In [5]:
results = pd.DataFrame(results, columns=['Methode', 'r2', 'RSME', 'Remaining NaNs', 'Time'])
results = results.set_index('Methode')
results

,r2,RSME,Remaining NaNs,Time
Methode,,,,
KNN with n=2,0.901304,0.125870,0,23.577335
KNN with n=3,0.955167,0.084835,0,24.117573
KNN with n=4,0.970061,0.069326,0,25.973104
KNN with n=5,0.966183,0.073679,0,26.145595
KNN with n=6,0.954115,0.085824,0,27.179772
KNN with n=7,0.915081,0.116755,0,26.696175
KNN with n=8,0.868175,0.145470,0,27.164371
KNN with n=9,0.828095,0.166118,0,26.457653
KNN with n=10,0.791541,0.182930,0,26.555461
